In [1]:
import sqlite3
import pandas as pd
from urllib.parse import urlparse, parse_qs

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_colwidth', 100)

In [66]:
DB_FILE = "../data/db/products.db"
conn = sqlite3.connect(DB_FILE)

df = pd.read_sql_query("SELECT * FROM product_details", conn)

conn.close()


In [8]:
# updates for database and webscraping code
# TODO brand_source_id is not working - null
# TODO remove "root" from category cols, reverse order of string inserted into db 

# data cleaning
# TODO clean size column 



In [67]:
df.columns

Index(['id', 'target_url', 'product_code', 'loves_count', 'rating', 'reviews',
       'brand_source_id', 'category_root_id', 'category_root_name',
       'category_root_url', 'sku_id', 'brand_name', 'display_name',
       'ingredients', 'limited_edition', 'first_access', 'limited_time_offer',
       'new_product', 'online_only', 'few_left', 'out_of_stock', 'price',
       'max_purchase_quantity', 'size', 'type', 'url', 'variation_type',
       'variation_value', 'returnable', 'finish_refinement', 'size_refinement',
       'created_at'],
      dtype='object')

In [77]:
def clean_compressed_product_hierarchy(df, col, delimiter=' --- ', code_prefix_to_strip='cat'):
    clean_col = df[col].str.replace(code_prefix_to_strip,"")
    clean_col = clean_col.str.split(delimiter)
    clean_col = clean_col.apply(lambda x : x[::-1])

    return pd.DataFrame(clean_col.to_list(), columns=[f'{col}_l1', f'{col}_l2', f'{col}_l3'])


def parse_parent_code_from_url(url):
    parsed_url = urlparse(url)
    params = parse_qs(parsed_url.query)
    return params.get('parentProduct', [None])


In [69]:
df = pd.concat([
    df, 
    clean_compressed_product_hierarchy(df, 'category_root_id', delimiter=' --- ', code_prefix_to_strip='cat'),
    clean_compressed_product_hierarchy(df, 'category_root_name', delimiter=' --- ', code_prefix_to_strip=''),
    clean_compressed_product_hierarchy(df, 'category_root_url', delimiter=' --- ', code_prefix_to_strip='/shop/')
], axis=1)

df = df.drop(['category_root_id', 'category_root_name', 'category_root_url'], axis=1)

df['parent_product_code'] = df['url'].apply(lambda x : parse_parent_code_from_url(x)[0])

df['price'] = df['price'].str.strip('$').astype(float)


In [71]:
df['category_root_name_l1'].value_counts()

category_root_name_l1
Makeup             15243
Skincare            2620
Fragrance           2208
Hair                1655
Bath & Body          516
Mini Size            360
Tools & Brushes       35
Gifts                 34
Men                   11
Name: count, dtype: int64

In [78]:


# parentProduct=P512478

In [80]:
df['parent_product_code'].value_counts()

parent_product_code
P393401      138
P377873      118
P87985432    118
P378284      112
P468353       94
            ... 
P504886        1
P428956        1
P474872        1
P243320        1
P506156        1
Name: count, Length: 2425, dtype: int64

0         47.0
1         46.0
2         65.0
3         48.0
4         53.0
         ...  
22677     43.5
22678    127.0
22679     43.5
22680     34.0
22681     43.5
Name: price, Length: 22682, dtype: float64